<a href="https://colab.research.google.com/github/Aparimita18/Question_Answering_Model/blob/main/Question_Answering_Model_on_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMuPDF transformers gradio

PDF extraction

In [2]:
import fitz

book = fitz.open("/content/dataset.pdf")

# Extract text from all pages
data = ""
for page in book:
    data += page.get_text()

print(data[:1000])

Speech and Language Processing
An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition
with Language Models
Third Edition draft
Daniel Jurafsky
Stanford University
James H. Martin
University of Colorado at Boulder
Copyright ©2024. All rights reserved.
Draft of January 12, 2025. Comments and typos welcome!
Summary of Contents
I
Fundamental Algorithms for NLP
1
1
Introduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
3
2
Regular Expressions, Tokenization, Edit Distance . . . . . . . . . . . . . . .
4
3
N-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
32
4
Naive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 56
5
Logistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 77
6
Vector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10

In [3]:
import fitz  # PyMuPDF
import pandas as pd

# Open PDF
doc = fitz.open("dataset.pdf")
page = doc[0]  # First page

# returns list of dicts with positional info
blocks = page.get_text("dict")["blocks"]

# Filter for text blocks
table_data = []
for b in blocks:
    if b["type"] == 0:  # text
        for line in b["lines"]:
            row = []
            for span in line["spans"]:
                row.append(span["text"].strip())
            if row:
                table_data.append(row)

# convert to DataFrame
for row in table_data:
    print(row)

df = pd.DataFrame(table_data)
print(df)


['Speech and Language Processing']
['An Introduction to Natural Language Processing,']
['Computational Linguistics, and Speech Recognition']
['with Language Models']
['Third Edition draft']
['Daniel Jurafsky']
['Stanford University']
['James H. Martin']
['University of Colorado at Boulder']
['Copyright ©2024. All rights reserved.']
['Draft of January 12, 2025. Comments and typos welcome!']
                                                    0
0                      Speech and Language Processing
1     An Introduction to Natural Language Processing,
2   Computational Linguistics, and Speech Recognition
3                                with Language Models
4                                 Third Edition draft
5                                     Daniel Jurafsky
6                                 Stanford University
7                                     James H. Martin
8                   University of Colorado at Boulder
9               Copyright ©2024. All rights reserved.
10  Draft of 

Text Cleaning

In [4]:
import re

def clean(text,debug= True):

    text = re.sub(r"-\n", "", text)
    # Replace multiple newlines with a single one
    text = re.sub(r"\n{2,}", "\n\n", text)


    text = re.sub(r'(Fig(?:ure)?\.? ?\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', text)
    text = re.sub(r"\(?[Tt]he interested reader.*?Section \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?see (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?refer to (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?(as|as we) (see|saw) (in )?(Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r'^\s*[\*\-•]+.*$', '', text, flags=re.MULTILINE)
    #paragraphs = re.split(r'\n?(?=\d+(\.\d+)*\s+[^\n]+)', text)
    text = re.sub(r'^\s*\d+(\.\d+)*\s+[•\-–—]\s+.+?\d\s*$', '', text, flags=re.MULTILINE)




    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()

        # Regular expressions to handle various cases
        if re.match(r'^\d+\s+Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.match(r'^Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'\d+\s*[•\-–—]?', line) or re.fullmatch(r'[•\-–—]?\s*\d+', line):
            continue
        if re.fullmatch(r'\d+', line):
            continue
        if re.fullmatch(r'\d*\.\d+', line):
            continue
        if re.fullmatch(r'(Figure|Table|Section|Subsection)\s+\d+(\.\d+)*', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'^•\s.*\s[\d.]+$',line):
            continue
        if re.search(r"(Speech and Language Processing|Jurafsky|Martin|Draft of \d{4})", line, re.IGNORECASE):
            continue
        if re.match(r'^[>\-•*]?\s*[A-Z\s]+$', line):
            continue
        if re.fullmatch(r'[\d\s•\-–—]*', line):
            continue
        if re.search(r"(C HAPTER|Speech and Language Processing|Jurafsky|Martin|Draft of \d{4}|CHAPTER \d+|N- GRAM|SPEECH RECOGNITION)", line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]{3,}', line) or line.isupper():
            continue
        if re.search(r'(Figure|Fig\.|Table|Box|Image|Diagram)\s*\d+(\.\d+)?', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[\[\(]?(image|diagram|fig|figure|illustration)[\]\)]?', line, re.IGNORECASE):
            continue
        if len(re.findall(r'[A-Za-z]', line)) < 4:
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]', line):
            continue

        cleaned_lines.append(line)

    cleaned_text = "\n".join(cleaned_lines)

    return cleaned_text


In [5]:
def splitpara(text):
    paragraphs = []
    current = ""

# Define known footer patterns (can extend this as needed)
    footer_patterns = [
        r'^For example if we use both begin.*',
        r'^Speech and Language Processing.*',
        r'^Jurafsky.*',
        r'^Draft of \d{4}.*',
        r'^Page \d+.*',
        r'^Figure \d+.*',
        r'^.*\b\d{3,}\b.*$'  # Lines with page numbers
    ]

    # Clean up footers
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if any(re.match(pat, line.strip()) for pat in footer_patterns):
            continue
        cleaned_lines.append(line)

    # Join back into cleaned text
    cleaned_text = '\n'.join(cleaned_lines)

    # paragraph splitting based on subsection heading and punctuation
    lines = cleaned_text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Remove figure/image/table mentions
        line = re.sub(r'(Fig(?:ure)?\.?\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Table\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Box\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'\(?(see|refer to)?\s*(Fig(?:ure)?|Table|Box)\s*\d+(\.\d+)?\)?', '', line, flags=re.IGNORECASE)


        if re.match(r'^\d+(\.\d+)+\s+[A-Za-z]', line):
            if current.strip():
                paragraphs.append(current.strip())
                current = ""
            paragraphs.append(line.strip())
            continue

        # If line ends with a period
        if line.endswith('.'):
            current += " " + line
            paragraphs.append(current.strip())
            current = ""
        else:
            current += " " + line

    # Add remaining paragraph
        if current.strip():
          paragraphs.append(current.strip())

    #splitting on whitespace i.e 2 or more newlines
    final_paragraphs = []
    for para in paragraphs:
        chunks = re.split(r'\n\s*\n', para)
        final_paragraphs.extend([chunk.strip() for chunk in chunks if chunk.strip()])

    return final_paragraphs
    #return paragraphs

In [6]:
cleaned_text = clean(data)
paragraphs = splitpara(cleaned_text)

import textwrap
print(f"Total paragraphs: {len(paragraphs)}")
print(textwrap.fill(paragraphs[105]))

Total paragraphs: 25142
Exercises Vector Semantics and Embeddings Vector Semantics TF-IDF:
Weighing terms in the vector Word2vec Exercises Neural Networks
Training Neural Nets RNNs and LSTMs Summary: Common RNN NLP
Architectures The Transformer Parallelizing computation using a single
matrix X The input: embeddings for token and position The Language
Modeling Head 10 Large Language Models Sampling for LLM Generation
Pretraining Large Language Models Potential Harms from Language Models
11 Masked Language Models Fine-Tuning for Sequence Labelling: Named
Entity Recognition 12 Model Alignment, Prompting, and In-Context
Learning Model Alignment with Human Preferences: RLHF and DPO NLP
Applications 13 Machine Translation Bias and Ethical Issues Exercises
14 Question Answering, Information Retrieval, and RAG Information
Retrieval Answering Questions with RAG Exercises 15 Chatbots &
Dialogue Systems Properties of Human Conversation Exercises Feature
Extraction for ASR: Log Mel Spectrum ASR Ev

In [7]:
import pandas as pd

df = pd.DataFrame(paragraphs, columns=["text"])
df.to_csv("dataset.csv", index=False,sep="|", encoding="utf-8", header=True)
print("Saved as dataset.csv")

from google.colab import files
#files.download("dataset.csv")

Saved as dataset.csv


In [8]:
print(textwrap.fill(df['text'][1839]))

Smoothing, Interpolation, and Backoff There is a problem with using
maximum likelihood estimates for probabilities: any ﬁnite training
corpus will be missing some perfectly acceptable English word
sequences. That is, cases where a particular n-gram never occurs in
the training data but appears in the test set. Perhaps our training
corpus has the words ruby and slippers in it but just happens not to
have the phrase ruby slippers.


## **Re-check if the data extraction & cleaning is done properly**

In [9]:

#list of all paragraphs
paragraphs = df['text']
print(paragraphs)
print("\n")
print(len(paragraphs))

0          An Introduction to Natural Language Processing,
1        An Introduction to Natural Language Processing...
2        An Introduction to Natural Language Processing...
3        An Introduction to Natural Language Processing...
4        An Introduction to Natural Language Processing...
                               ...                        
25137    fragment of word, 13 Gaussian prior on weights...
25138    fragment of word, 13 Gaussian prior on weights...
25139    fragment of word, 13 Gaussian prior on weights...
25140    fragment of word, 13 Gaussian prior on weights...
25141    fragment of word, 13 Gaussian prior on weights...
Name: text, Length: 25142, dtype: object


25142


In [10]:
print(f"Total pages in PDF: {len(book)}")

Total pages in PDF: 599


In [11]:
print(f"Total paragraphs extracted: {len(paragraphs)}")

Total paragraphs extracted: 25142


In [12]:
print(textwrap.fill(df['text'][12530]))

These changes in air pressure obviously originate with the speaker and
are caused by the speciﬁc way that air passes through the glottis and
out the oral or nasal cavities. We represent sound waves by plotting
the change in air pressure over time.


In [13]:
import random

for _ in range(5):
    parano = random.randint(0, len(paragraphs)-1)
    print(f"--- Paragraph {parano} ---")
    print(textwrap.fill(paragraphs[parano]))
    print("\n\n")

--- Paragraph 18729 ---
Lexicon-based methods for Entity-Centric Affect What if we want to get
an affect score not for an entire document, but for a particular
affect lexicons with contextual embeddings to assign an affect score
to an entity in text. In the context of affect about people, they
relabel the Valence/Arousal/Dominance dimension as
Sentiment/Agency/Power. The algorithm ﬁrst trains classiﬁers to map
embeddings to scores:



--- Paragraph 10482 ---
Given this set of possible candidate translations Y, and some
similarity or alignment function util, we choose the best translation
ˆy as the translation which is most similar to all the other candidate
translations: ˆy = argmax util(y,c) Various util functions can be
used, like chrF or BERTscore or BLEU. We can get the



--- Paragraph 16419 ---
Let’s consider sample features for a feature-based classiﬁer (like
logistic regression or random forests), classifying the relationship
between American Airlines (Mention 1, or M1) and Tim

In [14]:
chapters = [
  "Introduction",
  "Regular Expressions, Tokenization, Edit Distance",
  "N-gram Language Models",
  "Naive Bayes, Text Classification, and Sentiment",
  "Logistic Regression",
  "Vector Semantics and Embeddings",
  "Neural Networks",
  "RNNs and LSTMs",
  "The Transformer",
  "Large Language Models",
  "Masked Language Models",
  "Model Alignment, Prompting, and In-Context Learning",
  "Machine Translation",
  "Question Answering, Information Retrieval, and RAG",
  "Chatbots & Dialogue Systems",
  "Automatic Speech Recognition and Text-to-Speech",
  "Annotating Linguistic Structure",
  "Sequence Labeling for Parts of Speech and Named Entities",
  "Context-Free Grammars and Constituency Parsing",
  "Dependency Parsing",
  "Information Extraction: Relations, Events, and Time",
  "Semantic Role Labeling",
  "Lexicons for Sentiment, Affect, and Connotation",
  "Coreference Resolution and Entity Linking",
  "Discourse Coherence",
]

In [15]:
!pip install fuzzywuzzy

In [16]:
from fuzzywuzzy import fuzz

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [17]:

matches = {}
for section in chapters :
    found = any(fuzz.partial_ratio(section.lower(), para.lower()) > 85 for para in paragraphs)

    #found = any(section.lower() in para.lower() for para in paragraphs)
    matches[section] = "✅ Found" if found else "❌ Not Found"

for k, v in matches.items():
    print(f"{k}: {v}")

Introduction: ✅ Found
Regular Expressions, Tokenization, Edit Distance: ✅ Found
N-gram Language Models: ✅ Found
Naive Bayes, Text Classification, and Sentiment: ✅ Found
Logistic Regression: ✅ Found
Vector Semantics and Embeddings: ✅ Found
Neural Networks: ✅ Found
RNNs and LSTMs: ✅ Found
The Transformer: ✅ Found
Large Language Models: ✅ Found
Masked Language Models: ✅ Found
Model Alignment, Prompting, and In-Context Learning: ✅ Found
Machine Translation: ✅ Found
Question Answering, Information Retrieval, and RAG: ✅ Found
Chatbots & Dialogue Systems: ✅ Found
Automatic Speech Recognition and Text-to-Speech: ✅ Found
Annotating Linguistic Structure: ✅ Found
Sequence Labeling for Parts of Speech and Named Entities: ✅ Found
Context-Free Grammars and Constituency Parsing: ✅ Found
Dependency Parsing: ✅ Found
Information Extraction: Relations, Events, and Time: ✅ Found
Semantic Role Labeling: ✅ Found
Lexicons for Sentiment, Affect, and Connotation: ✅ Found
Coreference Resolution and Entity Linki

##**Chunk text into tokens for BERT**

In [18]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [19]:

def chunk_paragraphs(paragraphs, tokenizer, max_length=512, stride=64):
    """
    Tokenizes and chunks paragraphs into BERT-compatible segments using a sliding window.
    Returns a list of dictionaries with input_ids, attention_mask, and original text.
    """
    chunked = []

    for idx, para in enumerate(paragraphs):
        encodings = tokenizer(
            para,
            return_overflowing_tokens=True,
            truncation=True,
            max_length=max_length,
            stride=stride,
            return_attention_mask=True,
            return_offsets_mapping=True,
            padding=False
        )

        for i in range(len(encodings["input_ids"])):
            chunk_text = tokenizer.decode(encodings["input_ids"][i], skip_special_tokens=True)
            chunked.append({
                "para_index": idx,
                "chunk_index": i,
                "input_ids": encodings["input_ids"][i],
                "attention_mask": encodings["attention_mask"][i],
                "text": chunk_text
            })

    return chunked


In [20]:
# Use your previously prepared paragraphs
paragraphs = df['text'].tolist()  # assuming df from earlier

# Tokenize and chunk
chunked_data = chunk_paragraphs(paragraphs, tokenizer)

# Example output
print(f"Total chunks: {len(chunked_data)}")
print("Sample chunk text:\n", chunked_data[10001]["text"])


Total chunks: 25951
Sample chunk text:
 yet languages also differ in many ways ( as has been pointed out since ancient times ; see understanding what causes such translation divergences translation divergence


Generate manual QA pairs

In [21]:
import pandas as pd

# Load paragraph dataset
paragraphs = pd.read_csv('/content/dataset.csv', sep = '|')
paragraphs = paragraphs.dropna().reset_index(drop=True)
#paragraphs.head()


In [22]:
'''# Create an empty DataFrame for manual QA
manual_qa = pd.DataFrame(columns=['question', 'answer'])
#example
new_row = pd.DataFrame([{'question': 'What is instruction tuning?',
                               'answer': 'Instruction tuning (short for instruction finetuning, and sometimes even shortened to instruct tuning) is a method for making an LLM better at following instructions. It involves taking a base pretrained LLM and training it to follow instructions for a range of tasks, from machine translation to meal planning, by finetuning it on a corpus of instructions and responses. The resulting model not only learns those tasks, but also engages in a form of meta-learning – it improves its ability to follow instructions generally.Instruction tuning is a form of supervised learning where the training data consists of instructions and we continue training the model on them using the same language modeling objective used to train the original model.'
                               }])

manual_qa = pd.concat([manual_qa, new_row], ignore_index=True)

# Save once done
#manual_qa.to_csv('manual.csv', index=False)
'''

"# Create an empty DataFrame for manual QA\nmanual_qa = pd.DataFrame(columns=['question', 'answer'])\n#example\nnew_row = pd.DataFrame([{'question': 'What is instruction tuning?',\n                               'answer': 'Instruction tuning (short for instruction finetuning, and sometimes even shortened to instruct tuning) is a method for making an LLM better at following instructions. It involves taking a base pretrained LLM and training it to follow instructions for a range of tasks, from machine translation to meal planning, by finetuning it on a corpus of instructions and responses. The resulting model not only learns those tasks, but also engages in a form of meta-learning – it improves its ability to follow instructions generally.Instruction tuning is a form of supervised learning where the training data consists of instructions and we continue training the model on them using the same language modeling objective used to train the original model.'\n                              

In [23]:
import pandas as pd
data = pd.read_csv('/content/manual.csv',sep = ',')
data

,question,answer
0,What is instruction tuning?,Instruction tuning (short for instruction fine...
1,Why did the parallelogram method receive more ...,parallelogram method received more modern atte...
2,Why is training more challenging in the mentio...,Training is trickier in the mention-ranking mo...
3,What is lemmatization in text normalization?,Lemmatization is the task of determining that ...
4,Why is lemmatization important for processing ...,Lemmatization is essential for processing morp...
...,...,...
154,What is the retriever/reader architecture in q...,"In the retriever/reader architecture, the retr..."
155,What is retrieval-augmented generation in the ...,Retrieval-augmented generation involves using ...
156,How can QA systems be evaluated when only a si...,QA systems can be evaluated by exact match wit...
157,What evaluation metric is used for free text a...,"For free text answers, QA can be evaluated usi..."


Generate automatic QA pairs

In [24]:

!pip install sentencepiece

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd
from tqdm import tqdm
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
!pip install transformers
!pip install sentencepiece


In [27]:
!pip install --upgrade  transformers sentencepiece

In [28]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install cloud-tpu-client
!pip install --upgrade torch_xla

Looking in indexes: https://download.pytorch.org/whl/cu118


In [29]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random

# Load tokenizer & model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
def generate_question_t5(context):
    input_text = "generate question: " + context.strip()
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    output_ids = model.generate(input_ids,
                                max_length=64,
                                num_beams=4,
                                early_stopping=True)

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [31]:
import pandas as pd

# Load your dataset
df = pd.read_csv("dataset.csv",sep='|')


sampled_df = df.sample(n=1000, random_state=42).reset_index(drop=True)


In [32]:
from transformers import pipeline

# Load the QA pipeline using a pre-trained BERT model
qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

def generate_answer_bert(context, question):

    result = qa_pipeline(question=question, context=context)
    return result['answer']


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [33]:

import json

qa_pairs = []

# Loop through sampled paragraphs
for para in tqdm(sampled_df['text']):
    try:
        question = generate_question_t5(para)
        answer = generate_answer_bert(para, question)
        qa_pairs.append({

            "question": question,
            "answer": answer
        })
    except Exception as e:
        print(f"Error: {e}")

# Save the output to JSON
with open("qa_pairs.json", "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

# Optionally, save to CSV
qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv("qa_pairs.csv", sep="|", index=False, encoding="utf-8")


 12%|█▏        | 122/1000 [03:23<11:28,  1.28it/s]

Error: `question` cannot be empty


 13%|█▎        | 133/1000 [03:38<14:02,  1.03it/s]

Error: `question` cannot be empty


 14%|█▍        | 144/1000 [03:46<06:51,  2.08it/s]

...

 88%|████████▊ | 875/1000 [21:34<01:18,  1.60it/s]

Error: `question` cannot be empty


 98%|█████████▊| 980/1000 [24:16<00:18,  1.08it/s]

Error: `question` cannot be empty


100%|██████████| 1000/1000 [24:36<00:00,  1.48s/it]


In [34]:

import json

qa_pairs = []

# Loop through sampled paragraphs
for para in tqdm(sampled_df['text']):
    try:
        question = generate_question_t5(para)
        answer = generate_answer_bert(para, question)
        qa_pairs.append({

            "question": question,
            "answer": answer
        })
    except Exception as e:
        print(f"Error: {e}")

# Save the output to JSON
with open("qa_pairs1.json", "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

# Optionally, save to CSV
qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv("qa_pairs1.csv", sep="|", index=False, encoding="utf-8")


 12%|█▏        | 122/1000 [03:11<09:20,  1.57it/s]

Error: `question` cannot be empty


 13%|█▎        | 133/1000 [03:26<11:16,  1.28it/s]

Error: `question` cannot be empty


 14%|█▍        | 144/1000 [03:34<06:55,  2.06it/s]

...

 88%|████████▊ | 875/1000 [21:05<01:18,  1.59it/s]

Error: `question` cannot be empty


 98%|█████████▊| 980/1000 [23:45<00:15,  1.25it/s]

Error: `question` cannot be empty


100%|██████████| 1000/1000 [24:06<00:00,  1.45s/it]


In [ ]:

import json

qa_pairs = []

# Loop through sampled paragraphs
for para in tqdm(sampled_df['text']):
    try:
        question = generate_question_t5(para)
        answer = generate_answer_bert(para, question)
        qa_pairs.append({

            "question": question,
            "answer": answer
        })
    except Exception as e:
        print(f"Error: {e}")

# Save the output to JSON
with open("qa_pairs2.json", "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

# Optionally, save to CSV
qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv("qa_pairs2.csv", sep="|", index=False, encoding="utf-8")


In [36]:

import json

qa_pairs = []

# Loop through sampled paragraphs
for para in tqdm(sampled_df['text']):
    try:
        question = generate_question_t5(para)
        answer = generate_answer_bert(para, question)
        qa_pairs.append({

            "question": question,
            "answer": answer
        })
    except Exception as e:
        print(f"Error: {e}")

# Save the output to JSON
with open("qa_pairs3.json", "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

# Optionally, save to CSV
qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv("qa_pairs3.csv", sep="|", index=False, encoding="utf-8")


 12%|█▏        | 122/1000 [03:11<09:16,  1.58it/s]

Error: `question` cannot be empty


 13%|█▎        | 133/1000 [03:26<11:30,  1.26it/s]

Error: `question` cannot be empty


 14%|█▍        | 144/1000 [03:35<06:34,  2.17it/s]

...

 88%|████████▊ | 875/1000 [21:15<01:36,  1.30it/s]

Error: `question` cannot be empty


 98%|█████████▊| 980/1000 [23:57<00:16,  1.20it/s]

Error: `question` cannot be empty


100%|██████████| 1000/1000 [24:17<00:00,  1.46s/it]


In [ ]:
num_batches = 5
batch_size = 4000

for i in range(num_batches):
    qa_pairs = []
    batch_df = sampled_df[i*batch_size : (i+1)*batch_size]

    for para in tqdm(batch_df['text'], desc=f"Batch {i}"):
        try:
            question = generate_question_t5(para)
            answer = generate_answer_bert(para, question)
            qa_pairs.append({
                "context" : para,
                "question": question,
                "answer": answer
            })
        except Exception as e:
            print(f"Error: {e}")

    # Save each batch
    with open(f"automatic{i}.json", "w", encoding="utf-8") as f:
        json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

    pd.DataFrame(qa_pairs).to_csv(f"automatic{i}.csv", sep="|", index=False, encoding="utf-8")


Batch 0:  12%|█▏        | 122/1000 [03:31<09:21,  1.56it/s]

Error: `question` cannot be empty


Batch 0:  13%|█▎        | 133/1000 [03:46<11:25,  1.26it/s]

Error: `question` cannot be empty


Batch 0:  14%|█▍        | 144/1000 [03:55<07:27,  1.91it/s]

Error: `question` cannot be empty


Batch 0:  20%|█▉        | 197/1000 [04:57<11:56,  1.12it/s]